In [1]:
import os
os.environ['SPARK_NAME'] = "/opt/spark"
# os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
# os.environ['PYSPARK_PYTHON'] = 'python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/opt/spark/work-dir/.venv/bin/python3'
os.environ['PYSPARK_PYTHON'] = '/opt/spark/work-dir/.venv/bin/python3'

# można też spróbować wykorzystać moduł findspark do automatycznego odnalezienia miejsca instalacji sparka
# import findspark
# findspark.init()
# lub
# findspark.init("/opt/spark")

In [2]:
from pyspark.sql import SparkSession

# spark = SparkSession.builder.master("spark://spark-master:7077").appName("Create-DataFrame").getOrCreate()
# konfiguracja z określeniem liczby wątków (2) oraz ilości pamięci do wykorzystania poza stertą interpretera Pythona
spark = SparkSession\
        .builder\
        .master("local[2]")\
        .appName("Create-DataFrame")\
        .config("spark.memory.offHeap.enabled","true")\
        .config("spark.memory.offHeap.size","4g")\
        .getOrCreate()
spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/16 13:25:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local[2] appName=Create-DataFrame>

In [3]:
# lista wartości zostaje podzielona na partycje i rozproszona na wszystkie dostępne węzły
# https://spark.apache.org/docs/3.5.3/api/python/reference/api/pyspark.SparkContext.parallelize.html
rdd = spark.sparkContext.parallelize(list(range(20)))

# rdd w formie rozproszonej zostaje scalone w listę zawierającą wszystkie elementy RDD
# np. za pomocą funkcji collect()
# https://spark.apache.org/docs/3.5.3/api/python/reference/api/pyspark.RDD.collect.html

rddCollect = rdd.collect()
display(type(rddCollect))
print(f"Liczba partycji: {rdd.getNumPartitions()}")
print(f"Pierwszy element: {rdd.first()}")
print(rddCollect)

list

Liczba partycji: 2
Pierwszy element: 0
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [4]:
# obiekt RDD może przechowywać dane z różnych źródeł, które są zgodne z systemem plików Apache Hadoop
# np. Amazon S3, Cassandra, HDFS, HBase i inne

# możemy dla uniknięcia potrzeby każdorazowego odwoływania się do kontekstu poprzez spark.sparkContext zapisać sobie to w zmiennej pomocniczej
sc = spark.sparkContext
# tutaj wczytamy do RDD plik tekstowy
pan_tadeusz_file = sc.textFile("pan-tadeusz.txt")

In [5]:
display(pan_tadeusz_file.getNumPartitions())
pan_tadeusz_file.take(10)

2

['Adam Mickiewicz',
 '',
 'Pan Tadeusz',
 'czyli ostatni zajazd na Litwie',
 '',
 'ISBN 978-83-288-2495-9',
 '',
 '',
 '',
 '']

In [6]:
# możemy zmienić liczbę automatycznie stworzonych partycji i ponownie rozproszyć je po węzłach
pan_tadeusz_file = pan_tadeusz_file.repartition(4)
pan_tadeusz_file.getNumPartitions()

# również metoda coalesce może posłużyć nam do zmiany ilości partycji dla obiektu RDD np. po zastosowaniu filtrowania, które
# znacznie zmniejsza wielkość pierwotnego obiektu RDD a co za tym idzie każdej partycji i dalsze obliczenia mogą nie być
# wykonywane zbyt efektywnie (zbyt mały rozmiar partycji)
# https://spark.apache.org/docs/3.5.3/api/python/reference/api/pyspark.RDD.coalesce.html
# główna różnica między repartition a coalesce jest taka, że ta pierwsza wykorzystuje mechanizm tasowania danych a ta druga może, ale nie
# musi go wykorzystywać gdyż możemy tym sterować za pomocą parametru wywołania tej metody

4

In [7]:
# jedną z funkcji dostępnej w tym API jest możliwość wykonania funkcji na każdej z partycji
# minusem może być to, że funkcja foreachPartition zwraca typ None, więc wyniki należy przetworzyć w inny sposób

def count_words(iterator):
    words = sum([len(x.split()) for x in iterator])
    print(words)

pan_tadeusz_file.foreachPartition(count_words)

17366e 3:>                                                          (0 + 2) / 2]
17129
17307
17293
                                                                                

In [8]:
# funkcje map oraz reduce

# możemy również wykonać operację w inny sposób, tym raze mapując funkcję na każdy element obiektu RDD
# zwrócony zostanie obiekt RDD, na którym możemy wykonać kolejne operacje
pan_tadeusz_file.map(lambda s: len(s.split())).take(10)

# np. reduce - i tu nawiązanie do dość znanej techniki przetwarzania, MapReduce
# więcej: https://en.wikipedia.org/wiki/MapReduce
# oraz: https://wiadrodanych.pl/big-data/jak-dziala-mapreduce/
pan_tadeusz_file.map(lambda s: len(s.split())).reduce(lambda a, b: a + b)

69095

In [9]:
# lub tak - ten sam efekt
from operator import add
pan_tadeusz_file.map(lambda s: len(s.split())).reduce(add)

69095

In [10]:
# różnica między map() a flatMap() dla tego przypadku
display(pan_tadeusz_file.map(lambda s: s.split()).take(10))
pan_tadeusz_file.flatMap(lambda s: s.split()).take(10)

[['I', 'zaraz', 'mogłem', 'pieszo,', 'do', 'Twych', 'świątyń', 'progu'],
 ['Iść', 'za', 'wrócone', 'życie', 'podziękować', 'Bogu),'],
 ['Tak', 'nas', 'powrócisz', 'cudem', 'na', 'Ojczyzny', 'łono.'],
 ['Tymczasem', 'przenoś', 'moją', 'duszę', 'utęsknioną'],
 ['Do', 'tych', 'pagórków', 'leśnych,', 'do', 'tych', 'łąk', 'zielonych,'],
 ['Szeroko', 'nad', 'błękitnym', 'Niemnem', 'rozciągnionych;'],
 ['Do', 'tych', 'pól', 'malowanych', 'zbożem', 'rozmaitem,'],
 ['Wyzłacanych', 'pszenicą,', 'posrebrzanych', 'żytem;'],
 ['Gdzie', 'bursztynowy', 'świerzop,', 'gryka', 'jak', 'śnieg', 'biała,'],
 ['Gdzie', 'panieńskim', 'rumieńcem', 'dzięcielina', 'pała,']]

['I',
 'zaraz',
 'mogłem',
 'pieszo,',
 'do',
 'Twych',
 'świątyń',
 'progu',
 'Iść',
 'za']